In [1]:
import torch
import torch.nn as nn
import torch.nn.init as init
import random
import copy
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
import DeepTreeEnsemble
import torch.nn.functional as F
from importlib import reload
import pandas as pd
reload(DeepTreeEnsemble)
from DeepTreeEnsemble import DeepTreeEnsemble, create_dataloaders_for_dataset 
# if this ^ doesn't work, try 'pip install -e .' in the root directory of it. Make sure you are in 
# the desired environment. 

In [2]:
train_dataloader, test_dataloader, input_size, output_size = create_dataloaders_for_dataset(dataset_name='breast_cancer', task='classification', batch_size=32)

xtrain size 455
xtest size 114
y_train size 455
y_test size 114


In [3]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_layers, output_size, dropout=.5):
        super(MLP, self).__init__()
        self.layers = nn.ModuleList()
        self.batch_norms = nn.ModuleList()

        # Create the first hidden layer
        self.layers.append(nn.Linear(input_size, hidden_layers[0]))
        self.batch_norms.append(nn.BatchNorm1d(hidden_layers[0]))

        # Create additional hidden layers
        for i in range(1, len(hidden_layers)):
            self.layers.append(nn.Linear(hidden_layers[i-1], hidden_layers[i]))
            self.batch_norms.append(nn.BatchNorm1d(hidden_layers[i]))

        # Output layer
        self.output = nn.Linear(hidden_layers[-1], output_size)

        # Dropout layer
        self.dropout = nn.Dropout(dropout)


    def forward(self, x):
        i = 0
        for layer, batch_norm in zip(self.layers, self.batch_norms):
            # x = F.relu(batch_norm(layer(x)))
            i=i+1
            x = self.dropout(x)
            # print('Layer dtype: ', layer.weight.dtype)
            x = x.to(torch.float32) # <-- area of concern
            # print('tensor dytpe: ', x.dtype)
            # layer.weight = layer.weight.to(torch.float32)
            x = F.relu(batch_norm(layer(x)))
            
            # print(f'dtype is {x.dtype} after layer {i}')

        x = self.output(x)
        return x

In [4]:
hidden_layers = [25, 25]
multiLayerPerceptron = MLP(input_size=input_size, hidden_layers=hidden_layers, output_size=output_size+1, dropout=.5) #hidden size default at 100

# Fill DTE params
modelSaveDirectory = './saved_models/'
DTEmlp2b = DeepTreeEnsemble(task_name='classification',
                                model_arch=multiLayerPerceptron,
                                model_dir=modelSaveDirectory, 
                                base_number=2,
                                epochs=10,
                                train_dataloader=train_dataloader, 
                                test_dataloader=test_dataloader,
                                learning_rate=0.0001)

In [7]:
DTEmlp2b.train_DTE()

|| TRAINING WEIGHT AVERAGING TREE ENSEMBLE ||
Task: classification
Randomly initializing 2 models
Model 1:


Training Process: 100%|██████████| 10/10 [00:00<00:00, 15.27it/s]


./saved_models/model_1.pth
Model 2:


Training Process: 100%|██████████| 10/10 [00:00<00:00, 15.48it/s]


./saved_models/model_2.pth
Model 4:


Training Process: 100%|██████████| 10/10 [00:00<00:00, 12.59it/s]

./saved_models/model_4.pth
 Maximum accuracy: 0.740017361111111, F1: 0.5925972929211831, Test: 1.0024913832545281, Train: 1.0216129032770793 
 



,DTE_acc,DTE_f1,DTE_test,DTE_train
0,0.740017,0.592597,1.002491,1.021613


In [8]:
DTEmlp2b.single_model()

|| TRAINING SINGLE MODEL || 
Task: Classification


Training Process:   0%|          | 0/30 [00:00<?, ?it/s]

Training Process: 100%|██████████| 30/30 [00:02<00:00, 13.39it/s]

 Maximum accuracy: 0.5819155092592593, F1: 0.5121738634420965, Test: 1.0226483131448427, Train: 1.0721386157141792 
 



,single_acc,single_f1,single_test,single_train
0,0.581916,0.512174,1.022648,1.072139


In [9]:
def run_experiment(DTE, trails, output_directory):
    results = pd.DataFrame()

    for i in range(trails):
        print(f'Trail {i}: \n')
        df1 = DTE.train_DTE()
        df2 = DTE.single_model()
        combined_df = pd.concat([df1, df2], axis=1)
        results = pd.concat([results, combined_df], axis=0)

    results.to_csv(output_directory, index=False)

In [10]:
output_directory = './fixed.csv'
run_experiment(DTEmlp2b, trails=200, output_directory=output_directory)

Trail 0: 

|| TRAINING WEIGHT AVERAGING TREE ENSEMBLE ||
Task: classification
Randomly initializing 2 models
Model 1:


Training Process:   0%|          | 0/10 [00:00<?, ?it/s]

Training Process: 100%|██████████| 10/10 [00:00<00:00, 13.83it/s]


./saved_models/model_1.pth
Model 2:


Training Process: 100%|██████████| 10/10 [00:00<00:00, 13.54it/s]


...

./saved_models/model_2.pth
Model 4:


Training Process: 100%|██████████| 10/10 [00:01<00:00,  7.73it/s]


./saved_models/model_4.pth
 Maximum accuracy: 0.523263888888889, F1: 0.5163095239303938, Test: 1.047775661945343, Train: 1.0708626226584117 
 

|| TRAINING SINGLE MODEL || 
Task: Classification


Training Process: 100%|██████████| 30/30 [00:04<00:00,  6.78it/s]

 Maximum accuracy: 0.5710358796296297, F1: 0.5774459152838695, Test: 1.044828895231088, Train: 1.027508322265413 
 



In [14]:
df = pd.read_csv("fixed.csv")

In [15]:
print(df.columns)

Index(['DTE_acc', 'DTE_f1', 'DTE_test', 'DTE_train', 'single_acc', 'single_f1',
       'single_test', 'single_train'],
      dtype='object')


In [16]:
print(f'single acc mean: {df["single_acc"].mean()}  DTE acc mean: {df["DTE_acc"].mean()}')
print(f'single acc var: {df["single_acc"].var()}  DTE acc var: {df["DTE_acc"].var()}')
print(f'single loss mean: {df["single_test"].mean()}  DTE loss mean: {df["DTE_test"].mean()}')
print(f'single loss var: {df["single_test"].var()}  DTE loss var: {df["DTE_test"].var()}')

single acc mean: 0.6639878472222221  DTE acc mean: 0.6907708333333333
single acc var: 0.021157894106093606  DTE acc var: 0.015036343502968324
single loss mean: 0.9942898472050825  DTE loss mean: 1.0222247399389746
single loss var: 0.0031114721879348575  DTE loss var: 0.0012182447499786237


: 

In [ ]:
hidden_layers = [25, 25]
multiLayerPerceptron = MLP(input_size=input_size, hidden_layers=hidden_layers, output_size=output_size+1, dropout=.5) #hidden size default at 100

# Fill DTE params
modelSaveDirectory = './saved_models/'
DTEmlp8b = DeepTreeEnsemble(task_name='classification',
                                model_arch=multiLayerPerceptron,
                                model_dir=modelSaveDirectory, 
                                base_number=4,
                                epochs=10,
                                train_dataloader=train_dataloader, 
                                test_dataloader=test_dataloader,
                                learning_rate=0.0001)

In [ ]:
output_directory = './outputBase8.csv'
run_experiment(DTEmlp8b, trails=5, output_directory=output_directory)